In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import torch
import os
import cv2
import albumentations as A
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.datasets as datasets
import torch
import torchvision.transforms as T
from torchvision.transforms import v2
from albumentations.pytorch import ToTensorV2
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import pandas as pd

2024-10-23 20:02:09.486786: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-23 20:02:09.495982: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-23 20:02:09.507320: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-23 20:02:09.510457: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-23 20:02:09.518665: I tensorflow/core/platform/cpu_feature_guar

In [2]:
df=pd.read_csv('./fingerprint_data')
print(df.shape)
df

(55265, 9)


,Unnamed: 0.1,Unnamed: 0,file_path,modification,gender,hand,finger,method,id
0,0,0,./archive/SOCOFing/Real/1__M_Left_thumb_finger...,0,0,1,0,0,1
1,1,1,./archive/SOCOFing/Real/1__M_Right_ring_finger...,0,0,0,3,0,1
2,2,2,./archive/SOCOFing/Real/1__M_Left_middle_finge...,0,0,1,2,0,1
3,3,3,./archive/SOCOFing/Real/1__M_Right_index_finge...,0,0,0,1,0,1
4,4,4,./archive/SOCOFing/Real/1__M_Left_index_finger...,0,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...
55260,55260,55260,./archive/SOCOFing/Altered/Altered-Hard/600__M...,3,0,0,3,1,600
55261,55261,55261,./archive/SOCOFing/Altered/Altered-Hard/600__M...,3,0,0,4,2,600
55262,55262,55262,./archive/SOCOFing/Altered/Altered-Hard/600__M...,3,0,0,4,1,600
55263,55263,55263,./archive/SOCOFing/Altered/Altered-Hard/600__M...,3,0,0,3,2,600


In [3]:
train=df[df['id']<500]
val=df[df['id'].between(500,550)]
test=df[df['id']>550]

In [4]:

# Define a simple residual block
class SimpleResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(SimpleResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # If in_channels != out_channels or stride != 1, adjust the residual connection
        self.adjust_residual = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride)
        self.use_adjust = (in_channels != out_channels or stride != 1)
    
    def forward(self, x):
        residual = x
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        
        # Adjust the residual if necessary
        if self.use_adjust:
            residual = self.adjust_residual(x)
        
        out += residual
        return F.relu(out)

# Define a smaller ResNet-like model with 3 residual blocks
class SmallResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(SmallResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        
        # Define 3 residual blocks
        self.block1 = SimpleResidualBlock(16, 32, stride=2)
        self.block2 = SimpleResidualBlock(32, 64, stride=2)
        self.block3 = SimpleResidualBlock(64, 128, stride=2)
        
        # Global average pooling and a fully connected layer
        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(128, num_classes)
    
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.global_pool(x)
        x = torch.flatten(x, 1)
        return self.fc(x)



In [5]:
class train_dataloader(torch.utils.data.Dataset):
    def __init__(self, frame):
        self.frame=frame
        self.front_path = [f for f in frame['file_path'].tolist()]
        self.pred = [a for a in frame['hand'].tolist()]

    def __getitem__(self, index):
        img_conv=img_trf_train(str(self.front_path[index]))
        image, target = img_conv, int(self.pred[index])
        return image, target

    def __len__(self):
        return len(self.frame)

class val_dataloader(torch.utils.data.Dataset):
    def __init__(self, frame):
        self.frame=frame
        self.front_path = [f for f in frame['file_path'].tolist()]
        self.pred = [a for a in frame['hand'].tolist()]

    def __getitem__(self, index):
        img_conv=img_trf_val(str(self.front_path[index]))
        image, target = img_conv, int(self.pred[index])
        return image, target

    def __len__(self):
        return len(self.frame)


In [6]:
def img_trf_train(code):
    try:
        img=cv2.imread(str(code))#'../datasets/prisoner/side_crop/'+str(code)+'.jpg')#frissiteni kell
        #max_side=max(img.shape[0],img.shape[1])
        #print(max_side)
        #add transform
        transform=A.Compose([
        A.HorizontalFlip(p=0.5),
        A.Affine(scale=(0.9,1.1), shear=(-5,5), rotate=(-10,10), p=0.3,fit_output=False,keep_ratio=True),
        A.GaussianBlur(blur_limit=(3, 7), sigma_limit=0, p=0.2),
        A.GridDistortion (num_steps=5, distort_limit=(-0.3, 0.3), interpolation=1, border_mode=2, p=0.2),
        A.RandomBrightnessContrast(p=0.4),
        A.Resize(112,112),
        ToTensorV2()])
        front_transformed=transform(image=img)["image"]
        return front_transformed 
        
    except:
        z=torch.zeros(3,112,112)
    return z

def img_trf_val(code):
    try:
        img=cv2.imread(str(code))#'../datasets/prisoner/side_crop/'+str(code)+'.jpg')#frissiteni kell
        #max_side=max(img.shape[0],img.shape[1])
        #print(max_side)
        #add transform
        transform=A.Compose([
        A.Resize(112,112),
        ToTensorV2()])
        front_transformed=transform(image=img)["image"]
        return front_transformed 
        
    except:
        z=torch.zeros(3,112,112)
    return z

In [7]:
train_dataset = train_dataloader(train)
val_dataset=val_dataloader(val)
test_dataset = val_dataloader(test)
train_dataloader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
val_dataloader= DataLoader(val_dataset, batch_size=1024, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=1024, shuffle=True)


In [10]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [11]:
import time
import copy
import gc

num_epochs = 20

def train_model():
    since = time.time()
    best_error = 100
    best_acc=0
    train_loss = []
    val_loss = []
    train_acc_history = []
    val_acc_history = []

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0
        running_corrects = 0
        total_train_samples = 0

        for images, masks in train_dataloader:
            images = images.to(device=device, dtype=torch.float)
            masks = masks.to(device=device, dtype=torch.long) 
            
            outputs = model(images)
            _, preds = torch.max(outputs, 1)  # Get predictions
            
            loss = loss_function(outputs, masks)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            running_corrects += (preds == masks).sum().item()  # Sum up correct predictions
            total_train_samples += masks.size(0)  # Count total samples in the batch

            train_loss.append(loss.item())

        epoch_train_loss = running_loss / len(train_dataloader)
        epoch_train_acc = running_corrects / total_train_samples
        train_acc_history.append(epoch_train_acc)

        # Validation phase
        model.eval()
        validation_loss = 0
        validation_corrects = 0
        total_val_samples = 0

        with torch.no_grad():
            for images, labels in val_dataloader:
                images = images.to(device=device, dtype=torch.float)
                labels = labels.to(device=device, dtype=torch.long) 

                output_val = model(images)
                val_loss_batch = loss_function(output_val, labels)
                validation_loss += val_loss_batch.item()

                _, val_preds = torch.max(output_val, 1)  # Get validation predictions
                validation_corrects += (val_preds == labels).sum().item()  # Sum up correct predictions
                total_val_samples += labels.size(0)  # Count total samples in the batch

        epoch_val_loss = validation_loss / len(val_dataloader)
        epoch_val_acc = validation_corrects / total_val_samples
        val_acc_history.append(epoch_val_acc)

        # Save best model based on validation loss
        if epoch_val_acc >= best_acc:
            best_acc=epoch_val_acc
            #best_error = epoch_val_loss
            best_epoch = epoch
            best_model = copy.deepcopy(model.state_dict())
            torch.save(best_model, './hand_predict_base.pt')
            print(f"Best Model Saved - Validation Accuracy: {best_acc} at Epoch: {best_epoch+1}")

        # Print epoch metrics
        print(f"Epoch [{epoch+1}/{num_epochs}], "
              f"Training Loss: {epoch_train_loss:.4f}, "
              f"Training Accuracy: {epoch_train_acc * 100:.2f}%, "
              f"Validation Loss: {epoch_val_loss:.4f}, "
              f"Validation Accuracy: {epoch_val_acc * 100:.2f}%")

        # Clean up
        gc.collect()
        with torch.no_grad():
            torch.cuda.empty_cache()

    time_elapsed = time.time() - since
    print(f"Training time was: {time_elapsed:.2f} seconds")
    print(f"Best Validation Accuracy: {best_acc} at Epoch: {best_epoch+1}")

    #return train_loss, val_loss, train_acc_history, val_acc_history


In [12]:
# Instantiate the model
model = SmallResNet(num_classes=10)
model.to(device)
loss_function = nn.CrossEntropyLoss() 

optimizer = optim.Adam(model.parameters(), lr=0.01)
#print(model)


In [ ]:
train_model()

Best Model Saved - Validation Accuracy: 0.5631668800682885 at Epoch: 1
Epoch [1/20], Training Loss: 0.6883, Training Accuracy: 61.22%, Validation Loss: 1.1666, Validation Accuracy: 56.32%
Best Model Saved - Validation Accuracy: 0.6717883055911225 at Epoch: 2
Epoch [2/20], Training Loss: 0.6101, Training Accuracy: 66.86%, Validation Loss: 0.6386, Validation Accuracy: 67.18%


In [ ]:
model_test= SmallResNet(num_classes=2)
loss_function = nn.CrossEntropyLoss()  # Use CrossEntropyLoss for segmentation
model_test.load_state_dict(torch.load('./hand_predict_base.pt',weights_only=True))
model_test.to(device)
model_test.eval()
print('Model is loadad for testing')

In [ ]:
model_test.eval()
validation_loss = 0
validation_corrects = 0
total_val_samples = 0
val_acc_history=[]
with torch.no_grad():
    for images, labels in test_dataloader:
        images = images.to(device=device, dtype=torch.float) 
        labels = labels.to(device=device, dtype=torch.long) 

        output_val = model_test(images)
        val_loss_batch = loss_function(output_val, labels)
        validation_loss += val_loss_batch.item()

        _, val_preds = torch.max(output_val, 1)  # Get validation predictions
        validation_corrects += (val_preds == labels).sum().item()  # Sum up correct predictions
        total_val_samples += labels.size(0)  # Count total samples in the batch

epoch_val_loss = validation_loss / len(val_dataloader)
epoch_val_acc = validation_corrects / total_val_samples
val_acc_history.append(epoch_val_acc)

In [ ]:
print(f"Test Accuracy: {epoch_val_acc}")
